# Extracting table header as list

In [1]:
import collections
collections.Callable = collections.abc.Callable

In [2]:
from bs4 import BeautifulSoup, NavigableString
from html_table_extractor.extractor import Extractor
import pandas as pd
import os
import re

In [3]:
path = "/home/riya/Downloads/ADT/Output_html_Chapter8/kaggle/working/output/"

In [4]:
os.chdir(path)

In [5]:
os.listdir()

['output_page_1.html',
 'output_page_2.html',
 'output_page_3.html',
 'output_page_4.html']

In [6]:
output_list = []

In [7]:
output_list

[]

In [8]:
allowed_two_letter_words = {'sl','sc', 'st', 'cc', 'tp', 'tb', 'ts', 'tg', 'km', 'gm', 'an', 'at', 'by', 'do', 'go', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'of', 'on', 'or', 'so', 'to', 'up', 'us'}

def replace_td_content(tag):
    if isinstance(tag, NavigableString):
        return  # Skip strings, as they don't have 'contents'

    if tag.name == 'td':
        # Extract only meaningful English words and numbers
        valid_characters = set("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-.%")
        filtered_content = ''.join(char if char in valid_characters else ' ' for char in tag.get_text())
        
        # Remove extra spaces and words with less than 3 characters
        cleaned_content = ' '.join(word for word in filtered_content.split() if len(word) >= 3 or word.isdigit() or word.lower() in allowed_two_letter_words or word=='%')
        
        # Set the cleaned content as the tag's string
        tag.string = cleaned_content
    elif tag.contents:
        for child in tag.contents:
            replace_td_content(child)


In [9]:
def process_table_cells(target_table):
    if not target_table:
        print("Table not found in the HTML content.")
        return

    flag1 = 0
    flag2 = 0

    # Loop through each row in the table
    for row in target_table.find_all('tr'):
        # Loop through each cell in the row
        if(flag1 == 1 and flag2 == 1):
            row.decompose()
        else : 
            for cell in row.find_all(['td', 'th']):
                # Process the cell content (you can modify this part based on your requirements)
                cell_content = cell.get_text(strip=True)
                if(cell_content == '1'):
                    flag1 = 1
                if(cell_content == '2'):
                    flag2 = 1        
            

In [10]:
# iterate through all file 
for file in os.listdir(): 
    # Check whether file is in text format or not 
    if file.endswith(".html"): 
        file_path = f"{path}{file}"
  
        with open(file_path) as fp:
            soup = BeautifulSoup(fp, "html.parser")
            
            for tag in soup.find_all(attrs={'style': True}):
                tag.attrs.pop('style')
            
            table_tag = soup.find('table')
            
            replace_td_content(table_tag)

            process_table_cells(table_tag)
            
            extractor1 = Extractor(table_tag)
            extractor1.parse()
            output_list.append(extractor1.return_list())
            

In [11]:
output_list

[[['ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs'],
  ['SL.No',
   'District',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks'],
  ['SL.No',
   'District',
   'No.of DCC Banks',
   'No.of membersLoantaken',
   'Agri-culture Loan',
   'Other Loan',
   'Total Loan',
   'Deposits'],
  ['1', '2', '464', '465', '466', '467', '4

In [12]:
list_to_remove = [[['ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs',
   'ration and Agricultural Marketing8.1 fl. .C.C. Banks on 31.3.2017. Rs.in lakhs'],
  ['SL.No',
   'District',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks'],
  ['SL.No',
   'District',
   'No.of DCC Banks',
   'No.of membersLoantaken',
   'Agri-culture Loan',
   'Other Loan',
   'Total Loan',
   'Deposits'],
  ['1', '2', '464', '465', '466', '467', '468', '469']]]

In [13]:
for list_set in output_list:
    for item in list_to_remove: 
        if(list_set == item):
            output_list.remove(list_set)

In [14]:
list_to_add = [[['8.1 D.C.C. Banks on 31-3-2017 Rs.in lakhs',
   '8.1 D.C.C. Banks on 31-3-2017 Rs.in lakhs',
   '8.1 D.C.C. Banks on 31-3-2017 Rs.in lakhs',
   '8.1 D.C.C. Banks on 31-3-2017 Rs.in lakhs',
   '8.1 D.C.C. Banks on 31-3-2017 Rs.in lakhs',
   '8.1 D.C.C. Banks on 31-3-2017 Rs.in lakhs',
   '8.1 D.C.C. Banks on 31-3-2017 Rs.in lakhs',
   '8.1 D.C.C. Banks on 31-3-2017 Rs.in lakhs'],
  ['SL.No',
   'District',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks',
   'DCC Banks'],
  ['SL.No',
   'District',
   'No.of DCC Banks',
   'No.of membersLoantaken',
   'Agri-culture Loan',
   'Other Loan',
   'Total Loan',
   'Deposits'],
  ['1', '2', '464', '465', '466', '467', '468', '469']]]

for item in list_to_add:
    output_list.append(item)


In [15]:
output_list

[[['8.2 KSCARD PLD Ban on 31.3.2017 Rs. in lakhs',
   '8.2 KSCARD PLD Ban on 31.3.2017 Rs. in lakhs',
   '8.2 KSCARD PLD Ban on 31.3.2017 Rs. in lakhs',
   '8.2 KSCARD PLD Ban on 31.3.2017 Rs. in lakhs',
   '8.2 KSCARD PLD Ban on 31.3.2017 Rs. in lakhs',
   '8.2 KSCARD PLD Ban on 31.3.2017 Rs. in lakhs',
   '8.2 KSCARD PLD Ban on 31.3.2017 Rs. in lakhs',
   '8.2 KSCARD PLD Ban on 31.3.2017 Rs. in lakhs'],
  ['',
   'ffl',
   'KSCARD PLD Banks',
   'KSCARD PLD Banks',
   'KSCARD PLD Banks',
   'KSCARD PLD Banks',
   'KSCARD PLD Banks',
   'KSCARD PLD Banks'],
  ['',
   'ffl',
   'No.of P.L.D. Banks',
   'No.of membersLoan taken',
   'Agri-cultureLoan',
   'Other Loan',
   'Total Loa',
   'nDeposits'],
  ['1', '2', '470', '471', '472', '473', '474', '475']],
 [['8.3 Credit Co-operative Societies on 31.3.2017.',
   '8.3 Credit Co-operative Societies on 31.3.2017.',
   '8.3 Credit Co-operative Societies on 31.3.2017.',
   '8.3 Credit Co-operative Societies on 31.3.2017.',
   '8.3 Credit Co

In [16]:
def add_space_before_year(input_string):
    # Regular expression to find a year in 'yyyy' format with an optional alphabet character before it
    pattern = re.compile(r'([a-zA-Z])?(\d{4})')
    
    # Function to add space between alphabet character and year
    def replace(match):
        alphabet_part, year_part = match.groups()
        if alphabet_part:
            return f'{alphabet_part} {year_part}'
        else:
            return year_part

    # Using re.sub() to apply the replacement function to the input string
    result = pattern.sub(replace, input_string)

    return result

In [17]:
def remove_before_all_occurrence(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'\b(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})\b')

    # Find all occurrences of the pattern in the input string
    matches = list(pattern.finditer(input_string))

    # If there are at least two occurrences of the pattern, remove everything before the second occurrence
    for i in range(len(matches)):
        second_occurrence_index = matches[i].start()
        input_string = input_string[second_occurrence_index:]

    return input_string.strip()

In [18]:
def convert_uppercase_to_camel_case(input_string):
    words = input_string.split()
    result_words = [word.capitalize() if word.isupper() else word for word in words]
    result_string = ' '.join(result_words)
    return result_string

In [19]:
def add_space_before_camelcase(input_string):
    # Use regular expression to find camel case words
    camelcase_pattern = re.compile(r'(?<=[a-z])(?=[A-Z])')
    result_string = camelcase_pattern.sub(' ', input_string)
    return result_string

In [20]:
def remove_extra_spaces(input_string):
    # Use regular expression to replace consecutive white spaces with a single space
    cleaned_string = re.sub(r'\s+', ' ', input_string)
    return cleaned_string.strip()

In [21]:
def add_space_before_word_with_condition(input_string):
    # Use regular expression to find words preceded by a number or special character
    pattern = re.compile(r'(?<=[%])\b(\w+)\b', flags=re.IGNORECASE)

    # Using re.sub() to add space before matched words
    result_string = pattern.sub(lambda x: ' ' + x.group(1), input_string)

    return result_string

In [22]:
def replace_words_with_blank(input_string, words_to_replace):
    # Constructing a regular expression pattern to match any of the words in the set
    # pattern = re.compile(r'\b(?:' + '|'.join(re.escape(word) for word in words_to_replace) + r')\b', flags=re.IGNORECASE)
    pattern = re.compile(r'(?<!\S)(?:' + '|'.join(re.escape(word) for word in words_to_replace) + r')(?!\S)', flags=re.IGNORECASE)

    # Using re.sub() to replace matched words with blank
    result = pattern.sub('', input_string)

    return result.strip()

In [23]:
words_to_replace = ['.CA', '.Dg', '.G.S', '.Jfl', '.Q.', '.ffl', '1 2', '1-10', '2 01', '2006 jAz 2016', '2011 twAi rAi AaP fiU', '2011 twAi zsj wAi EAzs %', '20112.2', '20112.6', '2011g', '24 7', '31.03.2017', '31.3.201', '31.3.2017', '31.3.2017.', '33 jAz 6', '6 jAz 1', '6 jAz 14', '6 wAU', 'A.ffl', 'A.g', 'AdP', 'Adg', 'Affl', 'Afi', 'Afli', 'Aid', 'Arg', 'Azs', 'CAU', 'CAZ', 'CAv', 'CAw', 'CAz', 'Cfi', 'CrP', 'Dfi', 'Dfl', 'E.J .n. .Dg .Jfl .JA.', 'EAf', 'Efi', 'Efl', 'JfiP', 'Jfl', 'KfiQ', 'MlM', 'Pfi', 'QAi', 'Raing', 'Rfl', 'SQA', 'Three', 'VAi', 'dfl', 'eng cy.U', 'ffl', 'fiAi', 'fiP', 'fiQ', 'fiQAi', 'fiU', 'fig', 'fl.', 'gLi', 'gMl', 'jAU', 'jAi', 'lion nit', 'ofD', 'rAi', 'tAwAi', 'tP', 'twAi', 'tzx', 'wAi', 'tzs']

In [24]:
def replace_words(input_string, word_replacements):
    # Constructing a regular expression pattern to match whole words in the set
    # pattern = re.compile(r'\b(?:' + '|'.join(re.escape(word) for word in word_replacements.keys()) + r')\b')
    pattern = re.compile(r'(?<!\S)(?:' + '|'.join(re.escape(word) for word in word_replacements.keys()) + r')(?!\S)')

    # Using re.sub() to replace matched words with their corresponding replacements
    result = pattern.sub(lambda x: word_replacements.get(x.group(), x.group()), input_string)

    return result

In [25]:
word_replacements = { "Ban on" : "Bank on 31-3-2017", "nDeposits" : "Deposits", "Societies on" : "Societies on 31-3-2017", "Cred it" : "Credit", "tHousing" : "Housing", "SocieAties son" : "Societies on 31-3-2017", "Loa" : "Loan", }

In [26]:
def process_data(input_data):
    for k in range(len(input_data)):
        for i in range(len(input_data[k])):
            for j in range(len(input_data[k][i])):
                #print(k,i,j,input_data[k][i][j])

                input_data[k][i][j] = add_space_before_year(input_data[k][i][j])
                input_data[k][i][j] = replace_words_with_blank(input_data[k][i][j], words_to_replace)
                input_data[k][i][j] = replace_words(input_data[k][i][j], word_replacements)
                input_data[k][i][j] = remove_before_all_occurrence(input_data[k][i][j])
                input_data[k][i][j] = add_space_before_camelcase(input_data[k][i][j])
                input_data[k][i][j] = convert_uppercase_to_camel_case(input_data[k][i][j])
                input_data[k][i][j] = remove_extra_spaces(input_data[k][i][j])
                input_data[k][i][j] = add_space_before_word_with_condition(input_data[k][i][j])

In [27]:
process_data(output_list)
output_list

[[['8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   '8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   '8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   '8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   '8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   '8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   '8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   '8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs'],
  ['',
   '',
   'Kscard Pld Banks',
   'Kscard Pld Banks',
   'Kscard Pld Banks',
   'Kscard Pld Banks',
   'Kscard Pld Banks',
   'Kscard Pld Banks'],
  ['',
   '',
   'No.of P.l.d. Banks',
   'No.of members Loan taken',
   'Agri-culture Loan',
   'Other Loan',
   'Total Loan',
   'Deposits'],
  ['1', '2', '470', '471', '472', '473', '474', '475']],
 [['8.3 Credit Co-operative Societies on 31-3-2017',
   '8.3 Credit Co-operative Societies on 31-3-2017',
   '8.3 Credit Co-operative Societies on 31-3-2017',
   '8.3 Credit Co-operative Societies on 31-3-2017',
   '8.3 Credit Co

In [28]:
#transpose the inner list
transp_list = [[list(pair) for pair in zip(*sublist)] for sublist in output_list]
transp_list

[[['8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs', '', '', '1'],
  ['8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs', '', '', '2'],
  ['8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   'Kscard Pld Banks',
   'No.of P.l.d. Banks',
   '470'],
  ['8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   'Kscard Pld Banks',
   'No.of members Loan taken',
   '471'],
  ['8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   'Kscard Pld Banks',
   'Agri-culture Loan',
   '472'],
  ['8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   'Kscard Pld Banks',
   'Other Loan',
   '473'],
  ['8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   'Kscard Pld Banks',
   'Total Loan',
   '474'],
  ['8.2 Kscard Pld Bank on 31-3-2017 Rs. in lakhs',
   'Kscard Pld Banks',
   'Deposits',
   '475']],
 [['8.3 Credit Co-operative Societies on 31-3-2017', '', '', '', '1'],
  ['8.3 Credit Co-operative Societies on 31-3-2017', '', '', '', '2'],
  ['8.3 Credit Co-operative Societies on 31-3-2017',
   'Credit Co-operative Soc

In [29]:
def find_table_details(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'\b(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})\b')

    # Find all occurrences of the pattern in the input string
    matches = pattern.finditer(input_string)

    # Initialize variables to store the matches
    match_positions = []

    # Store the positions of the matches
    for match in matches:
        match_positions.append(match.start())

    # If there is at least one occurrence of the pattern
    if match_positions:
        table_id_index = match_positions[0]
        input_string = input_string[table_id_index:]  
        space_index = input_string.find(' ') 
        table_id = input_string[table_id_index:space_index].strip()
        table_name = input_string[space_index:].strip()
        return table_id, table_name    
    else:
        return None, None


In [30]:
def convert_to_number(string):
    try:
        number = int(string)
    except ValueError:
        try:
            number = float(string)
        except ValueError:
            raise ValueError("Input string is not a valid number")
    return number

In [31]:
chapter_id = 8
chapter_name = "Co-operation and Agricultural Marketing"
#processing each column
def process_dictionary(input_data):
    df = pd.DataFrame(columns = ['Attr_id', 'Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description'])
    for inner_list in input_data:
        table_id, table_name = find_table_details(inner_list[0][0])
        previd=0
        for data_list in inner_list:
            i=0
            attr_id=0
            description = ''
            prev = ''
            for data in reversed(data_list):
                if data == '1' or data == '2':
                    break
                i = i+1;
                if i == 1 and data!='' and (previd==0 or convert_to_number(data)!=previd):
                    attr_id = convert_to_number(data)
                    previd = convert_to_number(data)
                if i > 1 and i < len(data_list):
                    if description == '':
                        description = data
                        prev = data
                    else : 
                        if prev != data and data!='':
                            description = ','.join([description, data]) #description + ' ' + data
                            prev = data
            if attr_id != 0 and description != "" :
                row = {'Attr_id' : attr_id, 'Chapter_id' : chapter_id, 'Chapter_name' : chapter_name, 'Table_id' : table_id, 'Table_name' : table_name, 'Description' : remove_extra_spaces(description)}
                df.loc[len(df)] = row
    df = df.sort_values(by=['Attr_id'])
    df.to_csv('/home/riya/Downloads/ADT/Output_html_Chapter8/Chapter8_Attributes.csv', sep=';', index=False)
    df.to_csv('/home/riya/Downloads/ADT/Dictionary/Chapter8_Attributes.csv', sep=';', index=False)
                                            

In [32]:
process_dictionary(transp_list)

In [ ]:
#Not used
def remove_before_second_occurrence(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'\b(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})\b')

    # Find all occurrences of the pattern in the input string
    matches = list(pattern.finditer(input_string))

    # If there are at least two occurrences of the pattern, remove everything before the second occurrence
    if len(matches) >= 2:
        second_occurrence_index = matches[1].start()
        input_string = input_string[second_occurrence_index:]

    return input_string.strip()